# Подготовка

### Клонируем nomeroff-net и устанавливаем все необходимые pip зависимости для него.

In [1]:
%%bash

yes | rm -r nomeroff-net
git clone https://github.com/ria-com/nomeroff-net.git
cd nomeroff-net
yes | rm -r .git
pip3 install -r requirements.txt

Cloning into 'nomeroff-net'...
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Загружаем модель детекта номерных знаков Mask-RCNN

In [2]:
%%bash

cd nomeroff-net/models
wget https://nomeroff.net.ua/models/mrcnn/mask_rcnn_numberplate_0640_2019_06_24.h5

--2019-06-24 15:06:18--  https://nomeroff.net.ua/models/mrcnn/mask_rcnn_numberplate_0640_2019_06_24.h5
Resolving nomeroff.net.ua (nomeroff.net.ua)... 31.28.161.85
Connecting to nomeroff.net.ua (nomeroff.net.ua)|31.28.161.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255858144 (244M) [application/octet-stream]
Saving to: ‘mask_rcnn_numberplate_0640_2019_06_24.h5’

     0K .......... .......... .......... .......... ..........  0% 4.27M 57s
    50K .......... .......... .......... .......... ..........  0% 7.31M 45s
   100K .......... .......... .......... .......... ..........  0% 11.8M 37s
   150K .......... .......... .......... .......... ..........  0% 10.6M 34s
   200K .......... .......... .......... .......... ..........  0% 10.8M 31s
   250K .......... .......... .......... .......... ..........  0% 9.13M 31s
   300K .......... .......... .......... .......... ..........  0% 11.5M 29s
   350K .......... .......... .......... .......... ..........  

### Клонируем Mask-RCNN - модель сегментации.

In [3]:
%%bash

cd nomeroff-net
git clone https://github.com/matterport/Mask_RCNN.git
cd Mask_RCNN
yes | rm -r .git
pip3 install -r requirements.txt

Cloning into 'Mask_RCNN'...
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Смотрим на входные данные.
Название изображения совпадает с самым крупным по полощади номером Грузинских форматов.

In [78]:
%%html
<img src="./images/GLG-037.jpg">

### Что бы создать датасет для тренировки OKR вырезаем плашки номерных знаков.

In [38]:
import os
import cv2
import numpy as np
import sys
import json
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import os
import shutil
from sklearn.model_selection import train_test_split

In [39]:
# указываем местоположение номерофф-нет
NOMEROFF_NET_DIR = os.path.abspath('./nomeroff-net')
sys.path.append(NOMEROFF_NET_DIR)

# указываем местоположение маск-рснн
MASK_RCNN_DIR = os.path.join(NOMEROFF_NET_DIR, 'Mask_RCNN')

# указываем папку для хранения логов мфак-рснн
MASK_RCNN_LOG_DIR = os.path.join(NOMEROFF_NET_DIR, 'logs')

# указываем путь к модели детекта плашки номерных знаков
MASK_RCNN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/mask_rcnn_numberplate_0640_2019_06_24.h5")

# Импортируем необходимые инструменты для нахождения плашки
from NomeroffNet import  filters, RectDetector, Detector

# Инициализируем инструменты и загружаем модель
nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)
nnet.loadModel(MASK_RCNN_MODEL_PATH)

rectDetector = RectDetector()

In [40]:
async def create_dataset(resultDir, fileList):
    '''
    Функция вырезает номерные плашки из указанного списка файлов fileList, 
    выравнивает их и складывает в папку resultDir,
    а так же создаёт json аннотации к каждому из изображений
    '''
    resultDirAnn = os.path.join(resultDir, "ann")
    
    os.mkdir(resultDirAnn)
    resultDirImg = os.path.join(resultDir, "img")
    os.mkdir(resultDirImg)

    for i, img_path in enumerate(fileList):
        print(i, "/", len(fileList), img_path)

        baseName = os.path.splitext(os.path.basename(img_path))[0]
        print(baseName)
        baseName = baseName.split("_")[0].replace(" ", "").replace("-", "")

        img = mpimg.imread(img_path)

        NP = nnet.detect([img])

        # Генерируем маски
        cv_img_masks = await filters.cv_img_mask_async(NP)

        # Находим 4 ключевые точки
        arrPoints = await rectDetector.detectAsync(cv_img_masks, outboundHeightOffset=0, fixGeometry=True, fixRectangleAngle=10)

        # Выравниваем перспекивное искажение и вырезаем зоны
        zones = await rectDetector.get_cv_zonesRGB_async(img, arrPoints)

        foundNumber = 0
        for zone in zones:
            res_path_img = os.path.join(resultDirImg, "{}_{}.png".format(baseName, foundNumber))
            res_path_ann = os.path.join(resultDirAnn, "{}_{}.json".format(baseName, foundNumber))
            res_ann = {
                    "description": baseName,
                    "name": "{}_{}".format(baseName, foundNumber),
                    "size": {
                        "height": zone.shape[0],
                        "width":  zone.shape[1]
                    }
                }


            if os.path.exists(res_path_img):
                foundNumber += 1
                continue

            # записываем аннотацию
            with open(res_path_ann, "w") as jsonFile:
                json.dump(res_ann, jsonFile)

            # записываем вырезанную плашку
            mpimg.imsave(res_path_img, zone)
            foundNumber += 1

In [41]:
rootDir   = './images'
resultDir = './dataset'

if os.path.exists(resultDir):
    shutil.rmtree(resultDir)
os.mkdir(resultDir)

testSize = 0.2
valSize  = 0.2

X = []
for dirName, subdirList, fileList in os.walk(rootDir):
    for i, fname in enumerate(fileList):
        X.append(os.path.join(dirName, fname))
        
# Разбиваем датасет на 3 выборки: тестовою, ренировочную и валидационную
X_train, X_test = train_test_split(X, test_size=testSize, random_state=20)
X_train, X_val = train_test_split(X_train, test_size=valSize, random_state=4)

for fileList, sub_dir in zip([X_train, X_test, X_val], ["train", "test", "val"]):
    resDir = os.path.join(resultDir, sub_dir)
    os.mkdir(resDir)
    print(resDir)
    await create_dataset(resDir, fileList)

./dataset/train
0 / 46 ./images/ZUR-395.jpg
ZUR-395
1 / 46 ./images/ZUR-222.jpg
ZUR-222
2 / 46 ./images/ZZU-827.jpg
ZZU-827
3 / 46 ./images/ZZG-645.jpg
ZZG-645
4 / 46 ./images/GLL-628.jpg
GLL-628
5 / 46 ./images/GLL-811.jpg
GLL-811
6 / 46 ./images/GLG-090.jpg
GLG-090
7 / 46 ./images/ZUR-287.jpg
ZUR-287
8 / 46 ./images/ZVZ-253.jpg
ZVZ-253
9 / 46 ./images/ZZZ-777_1.jpg
ZZZ-777_1
10 / 46 ./images/VRV-917.jpg
VRV-917
11 / 46 ./images/ZUZ-370.jpg
ZUZ-370
12 / 46 ./images/YZY-726.jpg
YZY-726
13 / 46 ./images/YVY-111.jpg
YVY-111
14 / 46 ./images/LYL-508.jpg
LYL-508
15 / 46 ./images/ZVZ-262.jpg
ZVZ-262
16 / 46 ./images/LUL-579.jpg
LUL-579
17 / 46 ./images/YVV-012.jpg
YVV-012
18 / 46 ./images/GLG-195.jpg
GLG-195
19 / 46 ./images/ZZZ-784.jpg
ZZZ-784
20 / 46 ./images/LYL-866.jpg
LYL-866
21 / 46 ./images/ZZL-192.jpg
ZZL-192
22 / 46 ./images/YZY-285.jpg
YZY-285
23 / 46 ./images/ZZG-552_1.jpg
ZZG-552_1
24 / 46 ./images/ZUR-914.jpg
ZUR-914
25 / 46 ./images/YYY-016.jpg
YYY-016
26 / 46 ./images/ZUR-817

### Тренировка на полученном датасете

In [42]:
# очищаем графф
import keras
keras.backend.clear_session()

In [43]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

DATASET_NAME = "ge"
MODE         = "cpu"

PATH_TO_DATASET   = "dataset"
RESULT_MODEL_PATH = "model.h5"

from NomeroffNet.Base import OCR

In [70]:
class kz(OCR):
    def __init__(self):
        OCR.__init__(self)
        
        # generate automaticly
        #self.letters = ["7"]
        
        self.EPOCHS = 1

In [71]:
ocrTextDetector = kz()
model = ocrTextDetector.prepare(PATH_TO_DATASET, aug_count=0)

GET ALPHABET
Max plate length in "val": 6
Max plate length in "train": 6
Max plate length in "test": 6
Letters train  {'G', '4', '9', 'Z', 'L', '5', '8', 'U', '1', '2', '7', '6', 'R', '3', 'Y', 'V', '0'}
Letters val  {'G', '4', '9', 'Z', '5', 'L', '8', '0', 'U', '2', '7', '6', 'R', '3', 'Y', 'V', '1'}
Letters test  {'G', '4', '9', 'Z', '5', 'L', '0', '8', 'U', '2', '6', '7', '3', 'Y', 'V', '1'}
Max plate length in train, test and val do match
Letters in train, val and test do match
Letters: 0 1 2 3 4 5 6 7 8 9 G L R U V Y Z

EXPLAIN DATA TRANSFORMATIONS
ge
Text generator output (data which will be fed into the neutral network):
1) the_input (image)
2) the_labels (plate number): GLG090 is encoded as [10, 11, 10, 0, 9, 0]
3) input_length (width of image that is fed to the loss function): 30 == 128 / 4 - 2
4) label_length (length of plate number): 6
START BUILD DATA
ge
ge
ge
DATA PREPARED


**Примечание:**
На этом этапе могут возникнуть ошибки:
<ul>
   <li><span style="color:red">Max plate length in train, test and val do not match</span> - в одной из выборок(тестовой, валидациоонной или тренировочной) присутствует присутствуют примеры номеров в которых больше символов чем в остальных выборках (нельзя обучив модель находить 8-ми символьные и 7-символьные номера найти 9-ти символьный)</li> 
   <li><span style="color:red">Letters in train, val and test do not match</span> - в валидационной тестовой и тренировочной выборках встречаются разные символы (нельзя обучив модель  распознавать номера состоящие из символов {"A", "8"} распознать коректно номер в котором присутствует так же буква "R") </li> 
</ul>

In [72]:
ocrTextDetector.load(RESULT_MODEL_PATH)

In [73]:
model = ocrTextDetector.train(mode=MODE, is_random=0)


START TRAINING
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_ge (InputLayer)       (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_ge[0][0]               
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
_____________________________________________________________________________________________

In [74]:
ocrTextDetector.test(verbose=True)


RUN TEST

Predicted: 		 Z
True: 			 ZZG896

Predicted: 		 Z
True: 			 ZZZ067

Predicted: 		 Z
True: 			 GLG320

Predicted: 		 Z
True: 			 ZUZ658

Predicted: 		 Z
True: 			 ZVZ801

Predicted: 		 Z
True: 			 ZVZ801

Predicted: 		 Z
True: 			 ZZZ900

Predicted: 		 Z
True: 			 GLL993

Predicted: 		 Z
True: 			 LUL639

Predicted: 		 Z
True: 			 ZUZ722

Predicted: 		 Z
True: 			 YYV133

Predicted: 		 YZ
True: 			 GLG320

Predicted: 		 Z
True: 			 ZZZ375

Predicted: 		 Z
True: 			 GLG320

Predicted: 		 Z
True: 			 GLL993

Predicted: 		 Z
True: 			 ZZU651

Predicted: 		 ZY
True: 			 ZZZ067

Predicted: 		 Z
True: 			 YVY898

Predicted: 		 Z
True: 			 LUL147

Predicted: 		 Z
True: 			 GLL993
acc: 0.0


In [75]:
ocrTextDetector.save(RESULT_MODEL_PATH, verbose=True)

SAVED TO model.h5


**Примечания:**
<ul>
    <li><a href="https://github.com/ria-com/nomeroff-net/blob/master/NomeroffNet/Base/OCR.py"> Больше о реализации модели.</a></li>
    <li> Можно добавить классификатор при формировании датасета с вырезанными номерными плашками, что бы не сохранять ложные срабатывания сегментатора. Как подключить классификатор можно посмотреть <a href="https://github.com/ria-com/nomeroff-net/blob/master/tools/avto-nomer-tool/py/avto-nomer_grab.ipynb">тут</a>.</li>
    <li> Можно натренировать модель на малом наборе данных, затем применить её к большому набору данных для авторазметки. <a htef="https://github.com/ria-com/nomeroff-net/blob/master/tools/avto-nomer-tool/py/ocr_dataset_checker.ipynb">Пример.</a></li>
</ul>
